# Custom view report per user

In [1]:
from arcgis.gis import GIS
from getpass import getpass
import pandas as pd
from pprint import pprint

In [2]:
gis = GIS('https://www.arcgis.com', 'tkinlaw', getpass())

## Search for content
The default `max_items` is 10. Be sure to change that. Sorting can be done with `sort_field` and `sort_order`. These rely on basic fields from the item. View is not one of them. 
We want views. This is a property of the item, and is not exposed to `search` or `advanced_search`

In [3]:
allContent = gis.content.search('', max_items=500, sort_field = 'type', sort_order="desc")
print(len(allContent))

205


c:\Users\tim10393\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kinlaw.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\tim10393\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kinlaw.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\tim10393\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kinlaw.maps.arcgis.com'. Adding certific

## Bring results into a DataFrame for sorting

### Sort all content by views

In [4]:
rows = []
for item in allContent:
    rows.append({
        "title": item.title,
        "item": item,
        "type": item.type,
        "numViews": item.numViews,
        "user": item.owner
    })
df = pd.DataFrame(rows)
df.sort_values(by="numViews", ascending=False).head(10)

,title,item,type,numViews,user
49,Analysis,"{'id': '05407f2d7dce47a0bd89641e00a53511', 'ow...",StoryMap Theme,882,tkinlaw
157,Aircraft_WFL1,"{'id': '2d20dc78add14bad8dc2a0d4d8ae5f40', 'ow...",Feature Service,427,tkinlaw
52,Coaching Day Logistics,"{'id': '5e1c4ffe2c564606a0eb8c188f0b4c8c', 'ow...",StoryMap,424,tkinlaw
55,Training the model,"{'id': '2c327ec1ba6d4534b518aec53b593b8a', 'ow...",StoryMap,382,tkinlaw
56,Exporting training samples,"{'id': '0e59c289163f4895999540c3338b94ad', 'ow...",StoryMap,330,tkinlaw
21,Aircraft,"{'id': '24be629489304834bcdef018b4312da3', 'ow...",Web Map,302,tkinlaw
51,Preparing imagery for use,"{'id': '61c161ab92a74887870327fd53cdb068', 'ow...",StoryMap,295,tkinlaw
48,KNOW ramp plan,"{'id': '5acb2e9d636940a7bdc964fe52cf57fb', 'ow...",StoryMap Theme,263,tkinlaw
54,Deep learning,"{'id': '4dc982736eb1451783b825c810c3077b', 'ow...",StoryMap,251,tkinlaw
57,Building detection exercise alternatives,"{'id': 'df7ce715b1c94e6e98029244f210cef3', 'ow...",StoryMap,248,tkinlaw


### Sort user content by views

In [5]:
userNames = [user.username for user in gis.users.search('')]
print(f'There are {len(userNames)} users')

There are 5 users


In [6]:
for user in userNames:
    userDF = df[df['user']==user].sort_values(by="numViews", ascending=False).head(5)
    print(user, ": ", userDF['title'].to_list())

JaneDoe0 :  []
JohnDoe748 :  ['S_USA_Wilderness', 'Bigfoot_Sightings', 'S_USA_Wilderness', 'US Wilderness Areas', 'Bigfoot_Sightings']
mhnudi :  ['New_house_photos']
Tester0 :  []
tkinlaw :  ['Analysis', 'Aircraft_WFL1', 'Coaching Day Logistics', 'Training the model', 'Exporting training samples']


### Clean up results through a Pandas DataFrame
Results are displayed sorted by views, grouped by users, in descending order from total views, then by title is tied.

#### First, sort the values by user, then views, then title

In [7]:
df_sorted = df.sort_values(['user', 'numViews', 'title'], ascending=[True, False, True])
df_sorted

,title,item,type,numViews,user
177,S_USA_Wilderness,"{'id': 'cd3c72e2b7fc4458ab4814fc4933b2a0', 'ow...",Feature Service,17,JohnDoe748
184,Bigfoot_Sightings,"{'id': '2e71e69b7a1342d3911ea7ba64e710c4', 'ow...",Feature Service,13,JohnDoe748
62,Bigfoot_Sightings,"{'id': 'a4893d3c358846edadf73bcc593e46c0', 'ow...",Shapefile,1,JohnDoe748
61,S_USA_Wilderness,"{'id': '05518289df9a4776970e3f6ff089a8da', 'ow...",Shapefile,1,JohnDoe748
40,US Wilderness Areas,"{'id': '2d3eb177ff7c401193a3959f3318a4d8', 'ow...",Web Map,1,JohnDoe748
...,...,...,...,...,...
132,Untitled survey_1,"{'id': '9da513f4c6d44c5f8b750aae418ef095', 'ow...",Form,0,tkinlaw
133,Untitled survey_2,"{'id': '73771bcd608c4b5f988defaf76e97a34', 'ow...",Form,0,tkinlaw
156,Untitled survey_2,"{'id': '26a621a80c6949c4a1c1a59862f5282d', 'ow...",Feature Service,0,tkinlaw
76,d9eacc9e-f24a-4f93-b091-385043c82994.crpk,"{'id': '10f45a29b852429d92df97730f171469', 'ow...",Replication Package,0,tkinlaw


#### Next, extract those top 5 items per user

In [8]:
top5_per_user = df_sorted.groupby('user', group_keys=False).head(5)
top5_per_user

,title,item,type,numViews,user
177,S_USA_Wilderness,"{'id': 'cd3c72e2b7fc4458ab4814fc4933b2a0', 'ow...",Feature Service,17,JohnDoe748
184,Bigfoot_Sightings,"{'id': '2e71e69b7a1342d3911ea7ba64e710c4', 'ow...",Feature Service,13,JohnDoe748
62,Bigfoot_Sightings,"{'id': 'a4893d3c358846edadf73bcc593e46c0', 'ow...",Shapefile,1,JohnDoe748
61,S_USA_Wilderness,"{'id': '05518289df9a4776970e3f6ff089a8da', 'ow...",Shapefile,1,JohnDoe748
40,US Wilderness Areas,"{'id': '2d3eb177ff7c401193a3959f3318a4d8', 'ow...",Web Map,1,JohnDoe748
120,New_house_photos,"{'id': '660b7bcc746a45b6a03c211d41023862', 'ow...",Image Collection,0,mhnudi
49,Analysis,"{'id': '05407f2d7dce47a0bd89641e00a53511', 'ow...",StoryMap Theme,882,tkinlaw
157,Aircraft_WFL1,"{'id': '2d20dc78add14bad8dc2a0d4d8ae5f40', 'ow...",Feature Service,427,tkinlaw
52,Coaching Day Logistics,"{'id': '5e1c4ffe2c564606a0eb8c188f0b4c8c', 'ow...",StoryMap,424,tkinlaw
55,Training the model,"{'id': '2c327ec1ba6d4534b518aec53b593b8a', 'ow...",StoryMap,382,tkinlaw


#### Next, count the total number of views from top 5 items per user

In [9]:
sum_views = (
    top5_per_user.groupby('user', as_index=False)['numViews']
    .sum()
    .rename(columns={'numViews': 'sumTop5Views'})
    .sort_values('sumTop5Views', ascending=False)
)
sum_views

,user,sumTop5Views
2,tkinlaw,2445
0,JohnDoe748,33
1,mhnudi,0


#### Lastly, order the DataFrame by the total view count

This code also re-defines the index for the DataFrame so it tracks the desired order. Be aware that the two users with no items are not included in the DataFrame.

In [10]:
ordered_users = sum_views['user']
result = (
    top5_per_user
    .set_index('user')
    .loc[ordered_users]
    .reset_index()
)
result

,user,title,item,type,numViews
0,tkinlaw,Analysis,"{'id': '05407f2d7dce47a0bd89641e00a53511', 'ow...",StoryMap Theme,882
1,tkinlaw,Aircraft_WFL1,"{'id': '2d20dc78add14bad8dc2a0d4d8ae5f40', 'ow...",Feature Service,427
2,tkinlaw,Coaching Day Logistics,"{'id': '5e1c4ffe2c564606a0eb8c188f0b4c8c', 'ow...",StoryMap,424
3,tkinlaw,Training the model,"{'id': '2c327ec1ba6d4534b518aec53b593b8a', 'ow...",StoryMap,382
4,tkinlaw,Exporting training samples,"{'id': '0e59c289163f4895999540c3338b94ad', 'ow...",StoryMap,330
5,JohnDoe748,S_USA_Wilderness,"{'id': 'cd3c72e2b7fc4458ab4814fc4933b2a0', 'ow...",Feature Service,17
6,JohnDoe748,Bigfoot_Sightings,"{'id': '2e71e69b7a1342d3911ea7ba64e710c4', 'ow...",Feature Service,13
7,JohnDoe748,Bigfoot_Sightings,"{'id': 'a4893d3c358846edadf73bcc593e46c0', 'ow...",Shapefile,1
8,JohnDoe748,S_USA_Wilderness,"{'id': '05518289df9a4776970e3f6ff089a8da', 'ow...",Shapefile,1
9,JohnDoe748,US Wilderness Areas,"{'id': '2d3eb177ff7c401193a3959f3318a4d8', 'ow...",Web Map,1
